<a href="https://colab.research.google.com/github/lmencisoe/CDA/blob/main/taller%204/Taller4_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa librerías

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Lee la data

In [2]:
df_1 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v1.json')
#df_2 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v2.json')

# Convierte a numérica la variable de Total Charges

In [4]:
df_1['TotalCharges'] = df_1['TotalCharges'].replace("", 0)
df_1['TotalCharges'] = df_1['TotalCharges'].astype(float)
#df_2['TotalCharges'] = df_2['TotalCharges'].replace("", 0)
#df_2['TotalCharges'] = df_2['TotalCharges'].astype(float)

# Extrae la data entrenamiento 1 (X y Y)

In [5]:
X_total = df_1.drop(['Churn', 'customerID'], axis=1)
df_1['target'] = np.where(df_1['Churn']!= 'Yes', 1, 0)
Y_total = df_1['target'].astype(float)

# Train y test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)

# Pipeline limpieza de datos

In [7]:
cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )

# Pipeline XGBoost

In [8]:
xgb_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", XGBClassifier(),
                ),
            ]
        )

 # Hiperparámetros de XGBoost

In [11]:
xgb_param_grid = {
    'classifier__learning_rate': np.arange(0.05, 1, 0.05),
    'classifier__max_depth': np.arange(3, 20, 1),
    'classifier__n_estimators': np.arange(50, 200, 50),
    'classifier__subsample': np.arange(0.05, 1, 0.05)
}

randomized_roc_auc_xgb = RandomizedSearchCV(estimator=xgb_pipeline, 
                                            param_distributions=xgb_param_grid,
                                            n_iter=100,
                                            scoring='roc_auc',
                                            cv=3, 
                                            verbose=1)

with tf.device('/device:GPU:0'): 
  randomized_roc_auc_xgb.fit(X_train, y_train)

print('Best AUC: ', randomized_roc_auc_xgb.best_score_)
#print('Estimator: ', randomized_roc_auc.best_estimator_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best AUC:  0.8436272745515557


In [12]:
randomized_roc_auc_xgb.best_params_

{'classifier__subsample': 0.2,
 'classifier__n_estimators': 50,
 'classifier__max_depth': 3,
 'classifier__learning_rate': 0.05}

In [13]:
joblib.dump(randomized_roc_auc_xgb.best_params_, "/content/drive/MyDrive/CDA_taller4/xgb_best_hp.pkl")

['/content/drive/MyDrive/CDA_taller4/xgb_best_hp.pkl']

**AUC Test**

In [29]:
predict_xgb = pd.Series(randomized_roc_auc_xgb.predict_proba(X_test)[:,1])
metrics.roc_auc_score(y_test, predict_xgb)

0.8453394381121484

In [30]:
y_pred_train = randomized_roc_auc_xgb.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

         0.0       0.69      0.50      0.58      1276
         1.0       0.84      0.92      0.88      3524

    accuracy                           0.81      4800
   macro avg       0.76      0.71      0.73      4800
weighted avg       0.80      0.81      0.80      4800



In [31]:
y_pred_test = randomized_roc_auc_xgb.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

         0.0       0.68      0.47      0.55       316
         1.0       0.83      0.92      0.87       884

    accuracy                           0.80      1200
   macro avg       0.76      0.69      0.71      1200
weighted avg       0.79      0.80      0.79      1200



# Pipeline Decision Tree

In [13]:
dt_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", DecisionTreeClassifier(),
                ),
            ]
        )

# Hiperparámetros de Decision Tree

In [14]:
dt_param_grid = {
    'classifier__max_depth': np.arange(3, 10, 1),
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__min_samples_split': np.arange(5, 50, 5)
}

# Perform RandomizedSearchCV
randomized_roc_auc_dt = RandomizedSearchCV(estimator= dt_pipeline, 
                                            param_distributions=dt_param_grid,
                                            n_iter=50,
                                            scoring='roc_auc',
                                            cv=3, 
                                            verbose=1)

# Fit the estimator
with tf.device('/device:GPU:0'): 
  randomized_roc_auc_dt.fit(X_train, y_train)

# Compute metrics
print('Best AUC: ', randomized_roc_auc_dt.best_score_)
#print('Estimator: ', randomized_roc_auc.best_estimator_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best AUC:  0.808573046691763


**AUC Test**

In [15]:
predict = pd.Series(randomized_roc_auc_dt.predict_proba(X_test)[:,1])
metrics.roc_auc_score(y_test, predict)

0.8176030270920442

In [16]:
y_pred_train = randomized_roc_auc_dt.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

         0.0       0.68      0.46      0.55      1276
         1.0       0.83      0.92      0.87      3524

    accuracy                           0.80      4800
   macro avg       0.75      0.69      0.71      4800
weighted avg       0.79      0.80      0.79      4800



In [17]:
y_pred_test = randomized_roc_auc_dt.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

         0.0       0.67      0.45      0.53       316
         1.0       0.82      0.92      0.87       884

    accuracy                           0.80      1200
   macro avg       0.74      0.68      0.70      1200
weighted avg       0.78      0.80      0.78      1200



# Pipeline del Random Forest

In [18]:
rf_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", RandomForestClassifier(),
                ),
            ]
        )

# Hiperparámetros del Random Forest

In [19]:
rf_param_grid = {
    'classifier__max_depth': np.arange(3, 10, 1),
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__n_estimators': np.arange(5, 100, 5)
}

# Perform RandomizedSearchCV
randomized_roc_auc_rf = RandomizedSearchCV(estimator= rf_pipeline, 
                                            param_distributions=rf_param_grid,
                                            n_iter=50,
                                            scoring='roc_auc',
                                            cv=3, 
                                            verbose=1)

# Fit the estimator
with tf.device('/device:GPU:0'): 
  randomized_roc_auc_rf.fit(X_train, y_train)

# Compute metrics
print('Best AUC: ', randomized_roc_auc_rf.best_score_)
#print('Estimator: ', randomized_roc_auc.best_estimator_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best AUC:  0.8417116844844758


In [20]:
predict = pd.Series(randomized_roc_auc_rf.predict_proba(X_test)[:,1])
metrics.roc_auc_score(y_test, predict)

0.8390962397617274

In [21]:
y_pred_train = randomized_roc_auc_rf.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

         0.0       0.70      0.48      0.57      1276
         1.0       0.83      0.93      0.88      3524

    accuracy                           0.81      4800
   macro avg       0.77      0.70      0.72      4800
weighted avg       0.80      0.81      0.80      4800



In [22]:
y_pred_test = randomized_roc_auc_rf.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

         0.0       0.68      0.45      0.54       316
         1.0       0.83      0.93      0.87       884

    accuracy                           0.80      1200
   macro avg       0.75      0.69      0.71      1200
weighted avg       0.79      0.80      0.79      1200



# **Guarda el mejor modelo**

In [41]:
joblib.dump(randomized_roc_auc_xgb.best_estimator_, '/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl')

['/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl']